In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_machine_learning.utils import algorithm_globals
from qiskit.primitives import StatevectorSampler as Sampler, StatevectorEstimator as Estimator

from qiskit_machine_learning.circuit.library import QNNCircuit
from qiskit_machine_learning.neural_networks import EffectiveDimension
from qiskit_machine_learning.neural_networks import SamplerQNN

from models.qiskit_models import serial, strongly_parallel, all_to_all_crz, all_to_all_rzz, strongly_crz, strongly_rzz


algorithm_globals.random_seed = 42
sampler = Sampler()
estimator = Estimator()

In [ ]:
def serial_mixed(degree, )

In [ ]:
qc_str_ser, weights, _, _ = serial(5, 2, 1)
qc_str_ser.draw(output='mpl')

In [ ]:
qc = QNNCircuit(
    feature_map=ZZFeatureMap(feature_dimension=4, entanglement='full', reps=2),
    ansatz=RealAmplitudes(4, entanglement='full', reps=2, flatten=True),
)
enc = ZZFeatureMap(feature_dimension=4, entanglement='full', reps=2)
enc.decompose().draw(output='mpl')
# qc.draw(output="mpl", style="clifford")

In [ ]:
def parity(x):
    return "{:b}".format(x).count("1") % 2

In [ ]:
serial_models = [serial(degree=10, trainable_blocks=l, scaling=1) for l in range(1, 10)]
strongly_entangling_parallel_models = [strongly_parallel(n_qubits=10, trainable_layers=l, scaling=1) for l in range(1, 8)]
strongly_entangling_crz_models = [strongly_crz(n_qubits=10, trainable_layers=l, scaling=1) for l in range(1, 10)]
strongly_entangling_rzz_models = [strongly_rzz(n_qubits=10, trainable_layers=l, scaling=1) for l in range(1, 10)]
all_to_all_crz_models = [all_to_all_crz(n_qubits=10, trainable_layers=l, scaling=1) for l in range(1, 5)]
all_to_all_rzz_models = [all_to_all_rzz(n_qubits=10, trainable_layers=l, scaling=1) for l in range(1, 8)]

varying_models = [serial_models, strongly_entangling_parallel_models, strongly_entangling_crz_models, strongly_entangling_rzz_models, all_to_all_crz_models, all_to_all_rzz_models]
n = [5000, 8000, 10000, 40000, 60000, 100000, 150000, 200000, 500000, 1000000]

In [ ]:
eff_dim_of_varying_models = []

for models in varying_models:
    for model in models:
        qc, weights, feature, name = model
        if models is strongly_entangling_parallel_models:
            weight_params = [param for layer in weights['W'] for vec in layer for param in vec]
            final_params = []
        elif models is serial_models:
            weight_params = [param for vec in weights['W'] for param in vec]
            final_params = []
        else:
            weight_params = [param for layer in weights['W'] for vec in layer for param in vec]
            final_params = [param for vec in weights['final'] for param in vec]
        input_data = [feature]
        qnn = SamplerQNN(
            circuit=qc,
            input_params=input_data,
            weight_params=weight_params + final_params,
            interpret=parity,
            output_shape=2,
            sparse=False,
            sampler=sampler
        )
        d = qnn.num_weights
        input_samples = algorithm_globals.random.normal(0, 1, size=(10, qnn.num_inputs))
        weight_samples = algorithm_globals.random.uniform(0, 1, size=(10, qnn.num_weights))
        global_ed = EffectiveDimension(qnn, weight_samples, input_samples)
        global_eff_dim = global_ed.get_effective_dimension(dataset_size=n)
        eff_dim_of_varying_models.append((global_eff_dim, d, name))
        print("Stats of {}: number of weights = {}, effective dimension = {}".format(name, d, global_eff_dim))

    with open("eff_dims/eff_dim_of_varying_models_10qubits.pkl", "wb") as f:
        pickle.dump(eff_dim_of_varying_models, f)


In [ ]:
with open("eff_dims/eff_dim_of_varying_models_10qubits.pkl", "wb") as f:
    pickle.dump(eff_dim_of_varying_models, f)

In [ ]:
%matplotlib inline

group_colors = {
    "serial": "tab:blue",
    "parallel": "tab:orange",
    "strongly_crz": "tab:green",
    "strongly_rzz": "tab:red",
    "all_to_all_crz": "tab:purple",
    "all_to_all_rzz": "tab:brown"
}

# Map group to models
model_groups = {
    "serial": serial_models,
    "parallel": strongly_entangling_parallel_models,
    "strongly_crz": strongly_entangling_crz_models,
    "strongly_rzz": strongly_entangling_rzz_models,
    "all_to_all_crz": all_to_all_crz_models,
    "all_to_all_rzz": all_to_all_rzz_models
}

# Create a reverse map from model name to group (assumes unique names!)
model_to_group = {}
for group_name, models in model_groups.items():
    for model in models:
        _, _, _, name = model
        model_to_group[name] = group_name

# Sort results
eff_dim_of_varying_models.sort(key=lambda x: max(np.array(x[0]) / x[1]), reverse=True)

# Plot
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')
n_grid = np.array(n)

# Keep track of which group has been added to legend
group_labels_plotted = set()

for ed, d, name in eff_dim_of_varying_models:
    norm_ed = np.array(ed) / d
    d_grid = np.full_like(n_grid, d)

    group = model_to_group.get(name, "unknown")
    color = group_colors.get(group, "gray")

    # Add label only once per group
    label = group if group not in group_labels_plotted else None
    group_labels_plotted.add(group)

    ax.plot(n_grid, d_grid, ed, label=label, color=color)

ax.set_xlabel("Number of data (n)")
ax.set_ylabel("Number of parameters (d)")
ax.set_zlabel("Normalized Global Effective Dimension")
ax.set_title("Effective Dimension vs Model Size and Dataset Size")
ax.legend(title="Model Groups", loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.tight_layout()
plt.show()
fig.savefig("plots/eff_dim_plot_10qubits.png", dpi=300, bbox_inches='tight')

In [ ]:
table_data = {}

for dims, d, name in eff_dim_of_varying_models:
    mean_dim = np.mean(dims)
    if name not in table_data:
        table_data[name] = {}
    table_data[name][d] = mean_dim
# for dims, d, name in eff_dim_of_models:
#     mean_dim = np.mean(dims)
#     if name not in table_data:
#         table_data[name] = {}
#     table_data[name][d] = mean_dim

# mean_dim = np.mean(global_eff_dim_1)
# table_data[name1] = {}
# table_data[name1][d1] = mean_dim

# Convert to DataFrame
df = pd.DataFrame.from_dict(table_data, orient='index')
df.columns.name = 'Number of parameters (d)'
df.index.name = 'Model'

df = df[sorted(df.columns, reverse=False)]

display(df.style.format("{:.2f}", na_rep='–'))

In [ ]:
table_data = {}

for dims, d, name in eff_dim_of_varying_models:
    mean_dim = np.mean(dims)
    if name not in table_data:
        table_data[name] = {}
    table_data[name][d] = mean_dim
# for dims, d, name in eff_dim_of_models:
#     mean_dim = np.mean(dims)
#     if name not in table_data:
#         table_data[name] = {}
#     table_data[name][d] = mean_dim

# mean_dim = np.mean(global_eff_dim_1)
# table_data[name1] = {}
# table_data[name1][d1] = mean_dim

# Convert to DataFrame
df = pd.DataFrame.from_dict(table_data, orient='index')
df.columns.name = 'Number of parameters (d)'
df.index.name = 'Model'

df = df[sorted(df.columns, reverse=False)]

display(df.style.format("{:.2f}", na_rep='–'))

In [ ]:
with open("eff_dims/eff_dim_of_varying_models3.pkl", "rb") as f:
    eff_dim_of_varying_models = pickle.load(f)

In [ ]:
models = [strongly_parallel(n_qubits=5, trainable_layers=l, scaling=1) for l in range(1, 4)]

n = [5000, 8000, 10000, 40000, 60000, 100000, 150000, 200000, 500000, 1000000]
eff_dim_of_models = []

for model in models:
    qc, weights, feature, name = model
    weight_params = [param for layer in weights['W'] for vec in layer for param in vec]
    # final_params = [param for vec in weights['final'] for param in vec]
    input_data = [feature]
    qnn = SamplerQNN(
        circuit=qc,
        input_params=input_data,
        weight_params=weight_params, # + final_params,
        interpret=parity,
        output_shape=2,
        sparse=False,
        sampler=sampler
    )
    d = qnn.num_weights
    input_samples = algorithm_globals.random.normal(0, 1, size=(10, qnn.num_inputs))
    weight_samples = algorithm_globals.random.uniform(0, 1, size=(10, qnn.num_weights))
    global_ed = EffectiveDimension(qnn, weight_samples, input_samples)
    global_eff_dim = global_ed.get_effective_dimension(dataset_size=n)
    eff_dim_of_models.append((global_eff_dim, d, name))
    print("Stats of {}: number of weights = {}, effective dimension = {}".format(name, d, global_eff_dim))

In [ ]:
for ed in eff_dim_of_models:
    plt.plot(n, np.array(ed[0]) / ed[1], label=f"{ed[2]} {ed[1]}")

plt.xlabel("Number of data")
plt.ylabel("Normalized GLOBAL effective dimension")
plt.legend()
plt.show()

In [ ]:
%matplotlib notebook

eff_dim_of_models.sort(key=lambda x: max(np.array(x[0]) / x[1]), reverse=True)

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')

n_grid = np.array(n)

for ed, d, name in eff_dim_of_models:
    norm_ed = np.array(ed) / d
    d_grid = np.full_like(n_grid, d)
    ax.plot(n_grid, d_grid, norm_ed, label=f"{name} {d}")

ax.set_xlabel("Number of data (n)")
ax.set_ylabel("Number of parameters (d)")
ax.set_zlabel("Normalized Global Effective Dimension")
ax.set_title("Effective Dimension vs Model Size and Dataset Size")
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.tight_layout()
plt.show()
fig.savefig("plots/eff_dim_para.png", dpi=300, bbox_inches='tight')

In [ ]:
# circuit, weights, feature
# qc1, weights_dict1, feature_param1, name1 = serial(degree=5, scaling=1)

# Flatten parameters
# weight_params1 = [param for layer in weights_dict1['W'] for vec in layer for param in vec]
# final_params1 = [param for vec in weights_dict1['final'] for param in vec]
# weight_params1 = [param for vec in weights_dict1["W"] for param in vec]
# input_params1 = [feature_param1]

qnn1 = SamplerQNN(
    circuit=qc,
    # input_params=input_params1,
    # weight_params=weight_params1,
    interpret=parity,
    output_shape=2,  # +1 or -1 → 2-class classification
    sparse=False,
    sampler=sampler
)

d1 = qnn1.num_weights

input_samples1 = algorithm_globals.random.normal(0, 1, size=(10, qnn1.num_inputs))
weight_samples1 = algorithm_globals.random.uniform(0, 1, size=(10, qnn1.num_weights))

global_ed1 = EffectiveDimension(qnn=qnn1, weight_samples=weight_samples1, input_samples=input_samples1)

n = [5000, 8000, 10000, 40000, 60000, 100000, 150000, 200000, 500000, 1000000]
global_eff_dim_1 = global_ed1.get_effective_dimension(dataset_size=n)
print("Number of weights in {}: {}, norm_ed = {}".format("Abas", d1, global_eff_dim_1))
# plot the normalized effective dimension for the model
plt.plot(n, np.array(global_eff_dim_1) / d1, label="Abas")
plt.xlabel("Number of data")
plt.ylabel("Normalized GLOBAL effective dimension")
plt.legend()
plt.tight_layout()
plt.show()